In [ ]:
import numpy as np
import pandas as pd
import json
import pickle
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
from sklearn.metrics import jaccard_score

def recuperar_transacoes(i):
    filename = "transactions_all.txt"
    with open(filename, 'r') as file:
        transactions = [line.strip().split() for line in file.readlines()]
    return transactions

indice_arquivo = 0
transactions = recuperar_transacoes(indice_arquivo)


filtered_transactions0 = [transaction for transaction in transactions if transaction[-1] == '0']
filtered_transactions1 = [transaction for transaction in transactions if transaction[-1] == '1']
#add filtered transactionX for any number of classes in need and then put them on the filtered_transactions_list

filtered_transactions_list = [filtered_transactions0, filtered_transactions1]


def calculate_jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    similarity = len(intersection) / len(union)
    return similarity

def extract_association_rules(transactions, n_classes, rule_params,similar,supt):
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    df = pd.DataFrame(te_ary, columns=te.columns_)

    frequent_itemsets = fpgrowth(df, min_support=supt, use_colnames=True)
    rulesi = association_rules(frequent_itemsets, metric="lift", min_threshold=0)
    keep_indices = []

    for idx, antecedents in enumerate(rulesi['antecedents']):
        discard_rule = True 
        yi=[]
        for item in antecedents:
            if item.startswith('y'):
                yi_pattern = item.split('z')[0]
                yi.append(yi_pattern)
        if yi:
            occurrences = {pattern: yi.count(pattern) for pattern in yi}
            if not any(count == 1 for count in occurrences.values()):
                discard_rule = False
        if not discard_rule:
            keep_indices.append(idx)
            
    rules = rulesi.iloc[keep_indices]
    
    filtered_rules = rules[rules['consequents'].apply(lambda x: len(x) == 1 and isinstance(list(x)[0], str) and list(x)[0].isdigit())]
    
    new_rules = pd.DataFrame()
    for cls, (n_rules, min_antecedent_size) in enumerate(rule_params):
        class_rules = filtered_rules[filtered_rules['consequents'].apply(lambda x: str(cls) in x)]
        class_rules_with_min_antecedent = class_rules[class_rules['antecedents'].apply(lambda x: len(x) >= min_antecedent_size)]
        new_rules = pd.concat([new_rules, class_rules_with_min_antecedent])
        
    final_rules=pd.DataFrame()
    for cls, (n_rules, min_antecedent_size) in enumerate(rule_params):
        sim=True
        while sim:
            same_class_rule = new_rules[new_rules['consequents'].apply(lambda x: str(cls) in x)]
            top_same_class_rule = same_class_rule.nlargest(n_rules, 'lift')
            rep=[]
            for ij, i in enumerate(top_same_class_rule.iterrows()):
                if ij not in rep:
                    idx_i, rule_i = i
                    for jk, j in enumerate(top_same_class_rule.iterrows()):
                        if jk > ij and jk not in rep:
                            idx_j, rule_j = j
                            similarity = calculate_jaccard_similarity(set(rule_i['antecedents']), set(rule_j['antecedents']))
                            
                            if similarity>=similar:
                                rep.append(jk)
                                for idx, rule in new_rules.iterrows():
                                    if set(rule_j['antecedents']) == set(rule['antecedents']):
                                        new_rules = new_rules.drop(idx)
            if not rep:
                sim=False
                final_rules = pd.concat([final_rules, top_same_class_rule])
                

                                                     
        


    return final_rules



import pandas as pd

all_result_rules = pd.DataFrame()


n_classes = 2 #Number of classes
rule_params = [[75, 3], [75, 3]] #rule_params[0]->[maximum number of rules obtained for class 0, \
                                                  #minimum antecedent size for each rules from class 0]
similar = 0.3 #Jaccard Similarity amoung all antecedent groups must be below 30%
supt=[0.2,0.2] #[min support for class0, min support for class 1]
for i, transactions in enumerate(filtered_transactions_list):
    result_rules = extract_association_rules(transactions, n_classes, rule_params, similar,supt[i])
    
    all_result_rules = pd.concat([all_result_rules, result_rules])


count_consequents = all_result_rules['consequents'].value_counts()
print(count_consequents)


import pandas as pd

caminho_arquivo_pickle = 'result_rules0.pkl'
rulescsv='result_rules0.csv'

all_result_rules.to_pickle(caminho_arquivo_pickle)
all_result_rules.to_csv(rulescsv)